In [5]:
# MNIST 구현
# Tensorflow 구현
# PyTorch 구현
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime # 날짜 관련된 module, TensorBoard에서 이용한 로그를 저장할
                # 폴더명을 만들기 위해서 사용.

# 정규화를 편하게 하기 위해서
# 정규화 기법은 크게 2가지만 알아두면 되요!
# MinMaxScaler : 0 ~ 1 사이로 scaling -> 딥러닝 학습에 유리
# StandardScaler : 데이터의 분포를 유지하면서 값을 정규화, -3 ~ 3
# 일반적인 경우(머신러닝같은 경우)에는 표준화를 이용해서 정규화하는게 좋아요! 
# 딥러닝같은 경우 합습을 안정적으로 하기 위해 MinMaxScaler를 이용해서 정규화.
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

In [6]:
# 데이터 로딩과 전처리
df = pd.read_csv('./data/mnist/train.csv')

display(df)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# 결측치와 이상치는 없어요!
# 데이터를 분리
x_data = df.drop('label', axis=1, inplace=False).values # 2차원 ndarray
y_data = df['label'].values # 1차원 ndarray

# 학습데이터와 테스트데이터를 분리
x_data_train, x_data_test, y_data_train, y_data_test = \
train_test_split(x_data,
                 y_data,
                 test_size=0.2,
                 stratify=y_data,
                 random_state=42)

# 정규화
scaler_x = MinMaxScaler()
scaler_x.fit(x_data_train)

x_data_train_norm = scaler_x.transform(x_data_train)
x_data_test_norm = scaler_x.transform(x_data_test)

print(len(x_data_train_norm))

33600


In [ ]:
# keras 구현
keras_model = Sequential()

keras_model.add(Flatten(input_shape=(784,)))
keras_model.add(Dense(units=128,
                      activation='relu'))
keras_model.add(Dense(units=64,
                      activation='relu'))
keras_model.add(Dense(units=10,
                      activation='softmax'))

# 모델 설정
keras_model.compile(optimizer=Adam(learning_rate=1e-3),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

# Callback 설정
es_callback = EarlyStopping(monitor='val_loss',
                            patience=4,
                            restore_best_weights=True,
                            verbose=1)
# 저장하기 위한 ModelCheckpoint
# TensorBoard
log_dir = './logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tb_callback = TensorBoard(log_dir=log_dir,
                          histogram_freq=1)

# 모델 학습
keras_model.fit(x_data_train_norm,
                y_data_train,
                epochs=100,
                validation_split=0.2,
                callbacks=[es_callback, tb_callback],
                verbose=1,
                batch_size=32)
# Epoch 10/100
# 840/840 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step 
# - accuracy: 0.9955 - loss: 0.0148 - val_accuracy: 0.9674 - val_loss: 0.1397
# Epoch 10: early stopping
# Restoring model weights from the end of the best epoch: 6.

In [10]:
# PyTorch 구현
# Tensorflow 같은 경우 GPU설정(CUDA, cuDNN 설정)을 했다면 자동으로 GPU를 사용해서
# 학습을 진행!
# 하지만 PyTorch는 그렇지 않아요!
# 명시적으로 사용하는 Model과 Data를 GPU에 올려줘야 해요!
# PyTorch Lighting => GPU 설정이 되어 있다면 GPU에서 실행!
import torch
import torch.nn as nn # Layer
import torch.optim as optim # Optimizer
from torch.utils.tensorboard import SummaryWriter # TensorBoard

In [ ]:
# GPU를 사용할 수 있는지를 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [15]:
# TensorBoard를 위한 로그 파일
log_dir = './logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
writer = SummaryWriter(log_dir=log_dir)

# 데이터 변환
# 기존에 가지고 있던 ndarray를 PyTorch의 tensor로 변환
x_tensor_train = torch.FloatTensor(x_data_train_norm).to(device)
x_tensor_test = torch.FloatTensor(x_data_test_norm).to(device)

y_tensor_train = torch.LongTensor(y_data_train).to(device)
y_tensor_test = torch.LongTensor(y_data_test).to(device)

# Model을 만들어야 해요!
# Tensorflow같은 경우 Sequential 같은 것들을 이용해서 생성.
# PyTorch는 class를 생성해야 해요!
# 즉, 모델의 기능을 class로 정의한 후,
# class로부터 객체를 생성 => 우리의 모델 객체가 있어요!
# class는 모델의 기능을 설명!
# class의 instance(객체)가 실제 동작하는 모델이에요!
class MNISTModule(nn.Module): # nn.Module 클래스를 상속(inheritence)해서 우리 클래스를 정의해요!
    def __init__(self):       # class로부터 모델 객체가 생성될 때 자동으로 호출!
        super().__init__()    # 상위 클래스의 생성자를 호출해서 초기화를 진행!
        # 그 다음에는 우리 모델객체가 가지고 있는 Layer를 속성(변수)으로 명시
        self.hidden1 = nn.Linear(784, 128) # Tensorflow의 Dense Layer (입력과 출력을 동시에 명시)
        self.hidden2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 10)

    # 중요한 함수가 하나 나와요. 이 함수는 overriding 함수에요!
    # 순전파 기능을 하는 함수.
    def forward(self, x):
        # 위쪽에 명시해 놓은 Layer를 이용해서 순전파를 진행시키면 되요!
        # 우리 모델의 예측값을 return하면 되요!
        x = self.hidden1(x)
        x = nn.functional.relu(x)
        x = self.hidden2(x)
        x = nn.functional.relu(x)
        x = self.output(x)
        # softmax activation 처리는 여기서 하지 않아요!
        return x
    
# Model 객체 생성
torch_model = MNISTModule().to(device) # Model도 GPU에 올려야해요!

# Model을 만들었으니 나머지 설정에 관련된 것들을 만들어야 해요!
# Loss 지정
criterion = nn.CrossEntropyLoss() # Tensorflow에서 'categorical_crossentropy'
optimizer = optim.Adam(torch_model.parameters(), # 모델이 가지고 있는 weight
                       lr=1e-4)

# Model 학습
epochs = 10000
for epoch in range(epochs):
    torch_model.train() # Model 학습을 진행하겠다는 의미
                        # 이걸 해줘야 나중에 역전파를 위한 내부 그래프를 생성
    y_pred = torch_model(x_tensor_train) # 순전파를 진행해서 모델의 예측값을 도출
    loss = criterion(y_pred, y_tensor_train) # loss를 계산(숫자값을 계산 + 자료구조도 포함되요)

    # optimizer를 통해서 나중에 수정된 w값을 계산해야 해요!
    optimizer.zero_grad() # 가지고 있는 gradient를 0으로 초기화.
    loss.backward()       # 역전파를 진행(gradient만 계산해서 모든 가중치에 대한 gradient를 전파)
    optimizer.step()      # 역전파를 통해 update된 기울기를 이용해서 가중치를 수정!
    writer.add_scalar('Loss/train', loss.item(), epoch) # Log 파일에 기록!
    if epoch % 1000 == 0:
        print(f'Epochs : {epoch}/{epochs}')

Epochs : 0/10000
Epochs : 1000/10000
Epochs : 2000/10000
Epochs : 3000/10000
Epochs : 4000/10000
Epochs : 5000/10000
Epochs : 6000/10000
Epochs : 7000/10000
Epochs : 8000/10000
Epochs : 9000/10000


In [28]:
# 학습이 다 끝나면 평가를 진행
torch_model.eval() # 모델안에 있는 Dropout이나 BatchNormalization을 적용하지 않아요
with torch.no_grad(): # 속도를 높이기 위해서
    torch_y_pred = torch_model(x_tensor_test)
    # 확률값으로 나오니까 이 중 가장 높은 확률을 가지는 index가 숫자 이미지에요!
    torch_y_pred_class = torch.argmax(torch_y_pred, dim=1).cpu().numpy()
    result_accuracy = accuracy_score(y_tensor_test.cpu(), torch_y_pred_class)
    print(result_accuracy)

0.9639285714285715
